In [13]:
# a demo of training crossnet
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
import numpy as np
from models.crossnet import CrossNet
from preprocessing_utils import parse_delta

n = 100
ts_size = 10
static_size = 20
ts_len = 10
rnn_hid_size = 20
impute_weight = 0.3
epochs = 100

# generate masks, deltas from time-series input
ts_data = np.random.rand(n, ts_len, ts_size)
ts_masks = np.random.randint(2, size=ts_data.shape)
ts_deltas = parse_delta(ts_masks)

# generate masks from static input
static_data = np.random.rand(n, static_size)
static_masks = np.random.randint(2, size=static_data.shape)

labels = np.random.randint(2, size=(n,))

model = CrossNet(static_size, ts_size, rnn_hid_size, ts_len, impute_weight)

if torch.cuda.is_available():
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total params is {}'.format(total_params))

ts_data, ts_masks, ts_deltas, static_data, static_masks, labels = torch.FloatTensor(ts_data).cuda(), \
torch.FloatTensor(ts_masks).cuda(), torch.FloatTensor(ts_deltas).cuda(), torch.FloatTensor(static_data).cuda(), \
torch.FloatTensor(static_masks).cuda(), torch.FloatTensor(labels).cuda()

for epoch in range(epochs):
    model.train()
    ret = model.run_on_batch(ts_data, ts_masks, ts_deltas, static_data, static_masks, labels, optimizer, epoch)
print('loss:', ret['loss'])

Total params is 5811
loss: tensor(2.2629, device='cuda:0', grad_fn=<AddBackward0>)


In [14]:
# a demo of training bi-directional version of crossnet
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
import numpy as np
from models.bicrossnet import BiCrossNet
from preprocessing_utils import parse_delta

n = 100
ts_size = 10
static_size = 20
ts_len = 10
rnn_hid_size = 20
impute_weight = 0.3
epochs = 100

# generate masks, deltas from time-series input
ts_data = np.random.rand(n, ts_len, ts_size)
ts_masks = np.random.randint(2, size=ts_data.shape)
ts_deltas = parse_delta(ts_masks)

# generate reverse input for the bi-directional version
b_ts_data = np.array(ts_data[::-1,:,:])
b_ts_masks = np.array(ts_masks[::-1,:,:])
b_ts_deltas = parse_delta(b_ts_masks)

# generate masks from static input
static_data = np.random.rand(n, static_size)
static_masks = np.random.randint(2, size=static_data.shape)

labels = np.random.randint(2, size=(n,))

model = BiCrossNet(static_size, ts_size, rnn_hid_size, ts_len, impute_weight)

if torch.cuda.is_available():
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total params is {}'.format(total_params))

ts_data, ts_masks, ts_deltas, b_ts_data, b_ts_masks, b_ts_deltas, static_data, static_masks, labels = torch.FloatTensor(ts_data).cuda(), \
torch.FloatTensor(ts_masks).cuda(), torch.FloatTensor(ts_deltas).cuda(), torch.FloatTensor(b_ts_data).cuda(), \
torch.FloatTensor(b_ts_masks).cuda(), torch.FloatTensor(b_ts_deltas).cuda(), torch.FloatTensor(static_data).cuda(), \
torch.FloatTensor(static_masks).cuda(), torch.FloatTensor(labels).cuda()

for epoch in range(epochs):
    model.train()
    ret = model.run_on_batch(ts_data, ts_masks, ts_deltas, b_ts_data, b_ts_masks, b_ts_deltas, static_data, static_masks, labels, optimizer, epoch)
print('loss:', ret['loss'])

Total params is 11622
loss: tensor(4.4266, device='cuda:0', grad_fn=<AddBackward0>)
